# CBPP Federal Jobs Guarantee

This analyzes the [CBPP's 2018 Federal Jobs Guarantee proposal](https://www.cbpp.org/research/full-employment/the-federal-job-guarantee-a-policy-to-achieve-permanent-full-employment#_ftn1) from Mark Paul, William Darity, Jr., and Darrick Hamilton. It is primarily described by this table:

![img](https://imgur.com/5Km1yLO.png)

In addition to the \$32,500 average wage, the base wage is \$24,600. The program is available to people age 18 and over.

Modeling the uptake is nontrivial. I estimate each tax unit's benefit as follows:

1. Assign a JG wage $w$ which draws from a random uniform between \$24,600 and \$40,400.
2. Assign a maximum JG benefit $mb$ equal to $w * n_{18-64}$.
3. Assign an actual JG benefit $b$ equal to $max(\$0, mb - wages)$.
4. Assign % FTE (which includes both % FTE while working and share of the year with the JG job) equal to $b / (b+e00200)$ (`e00200` is "wages, salaries, and tips for filing unit"*).
5. Calculate the total FTE across tax units as the weighted sum of % FTE. This is the total FTE the JG would be expected to hire with 100% participation.
6. Divide the expected 9.7 million (from the table) by this FTE total. Call this $p$, the probability that each tax unit will participate in the JG.
7. Randomly assign each tax unit a participation flag with probability $p$.
8. Add $b$ to the `e00200` of tax units flagged as participating.
9. Multiply itemizable state and local income/sales taxes (`e18400`) by the change in `e00200p` (i.e., assume flat SALT--unrealistically).

\* This could be enhanced by splitting `e00200` between `e00200p` and `e00200s` for the taxpayer and spouse, respectively.

*Data: CPS  |  Tax year: 2018  |  Type: Static  |  Author: Max Ghenis*

## Setup

### Imports

In [1]:
import taxcalc as tc
import taxcalc_helpers as tch
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
tc.__version__

'0.20.1'

### Settings

In [3]:
sns.set_style('white')
DPI = 500
mpl.rc('savefig', dpi=DPI)
mpl.rcParams['figure.dpi'] = DPI
mpl.rcParams['figure.figsize'] = 6.4, 4.8  # Default.

In [4]:
mpl.rcParams['font.sans-serif'] = 'Roboto'
mpl.rcParams['font.family'] = 'sans-serif'

# Set title text color to dark gray (https://material.io/color) not black.
TITLE_COLOR = '#212121'
mpl.rcParams['text.color'] = TITLE_COLOR

# Axis titles and tick marks are medium gray.
AXIS_COLOR = '#757575'
mpl.rcParams['axes.labelcolor'] = AXIS_COLOR
mpl.rcParams['xtick.color'] = AXIS_COLOR
mpl.rcParams['ytick.color'] = AXIS_COLOR

# Use Seaborn's default color palette.
# https://stackoverflow.com/q/48958426/1840471 for reproducibility.
sns.set_palette(sns.color_palette())

In [5]:
# Specify number of decimals in tables.
pd.set_option('precision', 2)

## Generate data

### Load data

Generate a set of normal CPS records for 2018 using `Calculator`, then extract the dataframe.

In [6]:
calc = tc.Calculator(records=tc.Records.cps_constructor(), 
                     policy=tc.Policy(), verbose=False)
calc.advance_to_year(2018)

In [7]:
cps_raw_cols = pd.read_csv(
    os.path.join(tc.Records.CUR_PATH, 'cps.csv.gz')).columns

In [8]:
df = calc.dataframe(list(cps_raw_cols))

1) Assign a JG wage $w$ which draws from a random uniform between \$24,600 and \$40,400.

In [9]:
JG_MIN_WAGE = 24600
JG_AVG_WAGE = 32500
jg_max_wage = 39800  # Set to calibrate the average wage to $32,500.
# jg_max_wage = JG_AVG_WAGE + (JG_AVG_WAGE - JG_MIN_WAGE)
print(JG_MIN_WAGE, jg_max_wage)

24600 39800


In [10]:
df['jg_w'] = np.random.randint(low=JG_MIN_WAGE, high=jg_max_wage, 
                               size=df.shape[0])

In [11]:
print(df.jg_w.min(), df.jg_w.max())

24600 39799


2) Assign a maximum JG benefit $mb$ equal to $w * n_{1864}$.

In [12]:
def n65(df):
    return ((df.age_head >= 65).astype(int) + 
            (df.age_spouse >= 65).astype(int) + 
            df.elderly_dependent)

In [13]:
df['n65'] = n65(df)

In [14]:
df['n1864'] = df.n1820 + df.n21 - df.n65
df['jg_mb'] = df.jg_w * df.n1864

3) Assign an actual JG benefit $b$ equal to $max(\$0, mb - wages)$.

In [15]:
df['jg_b'] = np.maximum(0, df.jg_mb - df.e00200)

4) Assign % FTE (which includes both % FTE while working and share of the year with the JG job) equal to $n1864 * b / (b+e00200)$ (`e00200` is "wages, salaries, and tips for filing unit"*).

In [16]:
df['jg_fte'] = df.n1864 * df.jg_b / (df.jg_b + df.e00200)

In [17]:
df.jg_fte.mean()

0.4895278567011761

5) Calculate the total FTE across tax units as the weighted sum of % FTE. This is the total FTE the JG would be expected to hire with 100% participation.

In [18]:
total_potential_jg_fte = (df.jg_fte * df.s006).sum()
total_potential_jg_fte

71811317.37304085

6) Divide the expected 9.7 million (from the table) by this FTE total. Call this $p$, the probability that each tax unit will participate in the JG.

In [19]:
JG_FTE = 9700000
jg_p = JG_FTE / total_potential_jg_fte
jg_p

0.1350762018417105

7) Randomly assign each tax unit a participation flag with probability $p$.

In [20]:
df['jg_participate'] = np.random.rand(df.shape[0]) < jg_p

8) Add $b$ to the `e00200` of tax units flagged as participating, and split the additional amount between `e00200p` and `e00200s` according to the current split.

In [21]:
df['jg'] = df.jg_b * df.jg_participate

In [22]:
(df.jg * df.s006).sum() / 1e9

315.19119870177

In [23]:
df[df.jg_fte > 0][['e00200', 'jg', 'e00200p', 'jg_fte', 'n1864', 'jg_w', 'jg_mb', 'jg_b']].sort_values('e00200', ascending=False)

,e00200,jg,e00200p,jg_fte,n1864,jg_w,jg_mb,jg_b
98004,211638.82,0.00,0.00,0.54,6.0,38767,232602.0,20963.18
98012,211638.82,11093.18,0.00,0.30,6.0,37122,222732.0,11093.18
434972,176406.24,0.00,176406.24,0.47,5.0,38959,194795.0,18388.76
129727,171921.78,4273.22,171921.78,0.12,5.0,35239,176195.0,4273.22
81430,169739.80,3940.20,13400.08,0.11,5.0,34736,173680.0,3940.20
81424,169739.80,0.00,13400.08,0.14,5.0,34905,174525.0,4785.20
81425,169739.80,0.00,13400.08,0.60,5.0,38612,193060.0,23320.20
81423,169739.80,0.00,13400.08,0.44,5.0,37244,186220.0,16480.20
259638,163860.17,0.00,163860.17,0.34,5.0,35158,175790.0,11929.83
317386,162826.54,0.00,85171.18,0.33,5.0,34878,174390.0,11563.46


In [24]:
(df.jg * df.s006).sum() / (df.jg_participate * df.jg_fte * df.s006).sum()

32571.418079717852

In [25]:
df['e00200_orig'] = df.e00200
df['e00200p_orig'] = df.e00200p
df['e00200s_orig'] = df.e00200s

In [26]:
df['jgp'] = df.jg * np.where(df.e00200_orig > 0, 
                             df.e00200p_orig / df.e00200_orig, 1)
df['jgs'] = df.jg * np.where(df.e00200_orig > 0, 
                             df.e00200s_orig / df.e00200_orig, 0)

In [27]:
df['e00200'] = df.e00200_orig + df.jg
df['e00200p'] = df.e00200p_orig + df.jgp
df['e00200s'] = df.e00200s_orig + df.jgs

In [28]:
df.jgp.sum() / 1e6

731.584099501059

In [29]:
df.jgs.sum() / 1e6

121.45499329868814

In [30]:
df.jg.sum() / 1e6

853.0390927997471

In [31]:
df_noinc = df[(df.e00200_orig == 0)]
(df_noinc.jg * df_noinc.s006).sum() / 1e9

128.08371287645002

In [32]:
df_noinc = df[(df.e00200_orig > 0)]
(df_noinc.jg * df_noinc.s006).sum() / 1e9

187.10748582531997

In [33]:
df[['jg', 'jgp', 'jgs', 'e00200', 'e00200p', 'e00200s', 'n1864', 
    'e18400']].head()

,jg,jgp,jgs,e00200,e00200p,e00200s,n1864,e18400
0,0.0,0.0,0.0,48543.44,42211.89,6331.55,2.0,2647.39
1,0.0,0.0,0.0,16884.52,16884.52,0.00,1.0,920.03
2,0.0,0.0,0.0,0.00,0.00,0.00,0.0,679.67
3,0.0,0.0,0.0,23216.07,23216.07,0.00,1.0,1265.04
4,0.0,0.0,0.0,0.00,0.00,0.00,0.0,1060.34


People in tax units benefiting from JG.

In [34]:
((df.jg > 0) * df.s006 * df.n1864).sum() / 1e6

15.531844619999996

In [35]:
((df.jg > 0) * df.s006 * df.XTOT).sum() / 1e6

21.910523080000004

In [36]:
((df.e00200 * df.s006).sum() - (df.e00200_orig * df.s006).sum()) / 1e9

315.19119870176854

9) Multiply itemizable state and local income/sales taxes (`e18400`) by the change in `e00200p` (i.e., assume flat SALT--unrealistically).

**TODO**

### Clean data

Fix `e00900` pending https://github.com/open-source-economics/Tax-Calculator/issues/2024.

In [37]:
df['e00900'] = df.e00900p + df.e00900s

## Calculate

Run Tax-Calculator.

In [38]:
df.e00900.sum()

4823675338.633316

In [39]:
recs = tc.Records(data=df, 
                  start_year=2018, 
                  weights=tc.Records.CPS_WEIGHTS_FILENAME,
                  adjust_ratios=tc.Records.CPS_RATIOS_FILENAME,
                  benefits=tc.Records.CPS_BENEFITS_FILENAME)

In [40]:
jg = tch.calc_df(records=recs,
                 year=2018,
                 group_vars=['expanded_income', 'e00200', 'c00100'],
                 metric_vars=['aftertax_income', 'XTOT', 'nu18', 'eitc',
                              'e00200', 'c07220', 'c11070'])

In [41]:
base = tch.calc_df(records=tc.Records.cps_constructor(), #no_benefits=True),
                   year=2018,
                   group_vars=['expanded_income', 'e00200', 'c00100'],
                   metric_vars=['aftertax_income', 'XTOT', 'nu18', 'eitc',
                                'e00200', 'c07220', 'c11070'])

Add refundable and nonrefundable CTC components.

In [42]:
jg['ctc_m'] = jg.c07220_m + jg.c11070_m
base['ctc_m'] = base.c07220_m + base.c11070_m

Calculate differences across income measures, EITC, and CTC.

In [43]:
def jg_diff(var):
    diff = jg[var + '_m'].sum() - base[var + '_m'].sum()
    pct_diff = diff / base[var + '_m'].sum()
    print('JG changes ' + var + ' by ${:.1f}B'.format(diff / 1e3) +
          ' ({:.1f}%).'.format(pct_diff * 100))

In [44]:
jg_diff('e00200')
jg_diff('aftertax_income')
jg_diff('eitc')
jg_diff('ctc')

JG changes e00200 by $315.2B (3.9%).
JG changes aftertax_income by $255.0B (2.2%).
JG changes eitc by $-4.6B (-7.6%).
JG changes ctc by $2.6B (3.0%).


## Analysis

In [45]:
def add_poverty(df):
    EXTREME_POVERTY_LINE = 780
    df['pov_extreme_m'] = df.XTOT_m * (
        df.aftertax_income < (EXTREME_POVERTY_LINE * df.XTOT))
    df['pov_extreme_child_m'] = df.nu18_m * (
        df.aftertax_income < (EXTREME_POVERTY_LINE * df.XTOT))
    df['pov_10k_m'] = df.XTOT_m * (
        df.aftertax_income < (10000 * df.XTOT))
    df['pov_10k_child_m'] = df.nu18_m * (
        df.aftertax_income < (10000 * df.XTOT))
    df['fpl_m'] = df.XTOT_m * (df.c00100 < tch.fpl(df.XTOT))
    df['fpl_child_m'] = df.nu18_m * (df.c00100 < tch.fpl(df.XTOT))
    df['fpla_m'] = df.XTOT_m * (df.aftertax_income < tch.fpl(df.XTOT))
    df['fpla_child_m'] = df.nu18_m * (df.aftertax_income < tch.fpl(df.XTOT))

In [46]:
add_poverty(jg)
add_poverty(base)

In [49]:
def print_poverty(numerator, denominator='XTOT_m'):
    jg_rate = jg[numerator].sum() / jg[denominator].sum()
    base_rate = base[numerator].sum() / base[denominator].sum()
    chg = 1 - jg_rate / base_rate
    print('JG reduces ' + numerator + ' by {:.1f}%'.format(chg * 100) +
          ', from {:,.1f}%'.format(base_rate * 100) + 
          ' to {:,.1f}%.'.format(jg_rate * 100))

In [50]:
for i in ['fpla', 'fpl', 'pov_extreme', 'pov_10k']:
    print_poverty(i + '_m')
    print_poverty(i + '_child_m', 'nu18_m')

JG reduces fpla_m by 12.2%, from 8.8% to 7.7%.
JG reduces fpla_child_m by 9.9%, from 9.6% to 8.7%.
JG reduces fpl_m by 10.1%, from 26.9% to 24.2%.
JG reduces fpl_child_m by 11.5%, from 28.2% to 25.0%.
JG reduces pov_extreme_m by 12.4%, from 1.4% to 1.2%.
JG reduces pov_extreme_child_m by 9.7%, from 1.7% to 1.5%.
JG reduces pov_10k_m by 12.3%, from 14.6% to 12.8%.
JG reduces pov_10k_child_m by 11.0%, from 22.5% to 20.0%.
